In [0]:
dbutils.widgets.text("p_environment", "")

In [0]:
dbutils.widgets.get("p_environment")
v_environment = dbutils.widgets.get("p_environment")
print(v_environment)

## Ingestion del archivo country.json

### Paso 1- Leer el archivo JSON usando "DataFrameReader" de Spark

In [0]:
country_schema = "countryId INT, countryIsoCode STRING, countryName STRING"

In [0]:
countries_df = spark.read \
    .schema(country_schema) \
    .json("/mnt/moviehistoryl/bronce/country.json")

In [0]:
countries_df.printSchema()

In [0]:
display(countries_df)

### Paso 2 - Eliminar las columnas no deseadas

In [0]:
countries_dropped_df = countries_df.drop("countryIsoCode")

In [0]:
from pyspark.sql.functions import col

In [0]:
countries_dropped_df = countries_df.drop(col("countryIsoCode"))

In [0]:
countries_dropped_df = countries_df.drop(countries_df["countryIsoCode"])

### Paso 3 - Cambiar el nombre de las columnas y añadir "ingestion_date" y "evironment"

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
countries_final_df = countries_dropped_df.withColumnRenamed("countryName", "country_name") \
    .withColumnRenamed("countryId", "country_id")\
    .withColumn("ingestion_date", current_timestamp())\
    .withColumn("environment", lit(v_environment))


In [0]:
display(countries_final_df)

### Paso 4 - Escribir la salida en formato parquet

In [0]:
countries_final_df.write.mode("overwrite").parquet("/mnt/moviehistoryl/silver/countries")

In [0]:
spark.read.parquet("/mnt/moviehistoryl/silver/countries").display()

In [0]:
dbutils.notebook.exit("success")